<a href="https://colab.research.google.com/github/tvarog/kaldi-minilibrispeech-tutorial/blob/master/Kaldi_minilibrispeech_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Warsztaty z uczenia maszynowego A: problemy rozpoznawania mowy
## Automatyczne Rozpoznawania Mowy (ASR) w pigułce - tutorial Kaldi
###### 2020.05.15
---
###Marcin Sikora 
###(marcin.sikora at amu.edu.pl)
---

**Zaliczenia**:
1. Obecności
1. Zadania z 2-części warsztatów (2020-04-17, 2020-05-15)
1. Ocena końcowa to średnia ocena z 2-części


Termin zgłaszania zadań - do końca semestru.

Skala ocen:
- 3: zadanie 1.
- 4: zadanie 1, niedokończone zadanie 2. 
- 5: zadanie 1, 2.

**Agenda**:
1. Przygotowanie środowiska
1. Wprowadzenie teoretyczne
1. Trening modelu minilibrispeech
1. Zadania na zaliczenie

**Przydatne linki/literatura**:

- https://kaldi-asr.org/doc/kaldi_for_dummies.html
- https://kaldi-asr.org/doc/data_prep.html (jak przygotować dane do treningu/testów)
- https://github.com/tvarog/kaldi-minilibrispeech-tutorial (ten tutorial + zmodyfikowana recepta Kaldi
- Dan Jurafsky, James H. Martin, Speech and Language Processing (3rd ed. draft), https://web.stanford.edu/~jurafsky/slp3/
- Kaldi documentation, http://kaldi-asr.org/doc/ 
- Sanjeev Khudanpur, Dan Povey, Jan Trmal, Building Speech Recognition Systems with the Kaldi Toolkit, https://www.clsp.jhu.edu/wp-content/uploads/2016/06/Building-Speech-Recognition-Systems-with-the-Kaldi-Toolkit.pdf




![Kaldi](https://kaldi-asr.org/doc/KaldiTextAndLogoSmall.png)
##Czym jest Kaldi?
Kaldi to zestaw narzędzi open source napisany w C++, bashu, perlu i pythonie służący do budowania systemów rozpoznawania mowy i przetwarzania sygnałów, dostępny bezpłatnie na licencji Apache v2.0. Kaldi teoretycznie działa na Windowsie, ale w praktyce - nie polecam. Kaldi najlepiej czuje się na Linuksie, działa też po niewielkich modyfikacjach na OS X.

Za pomocą Kaldi możemy:
- zamienić sygnał mowy na cechy,
- wytrenować i/lub zaadaptować modele akustyczne i językowe,
- użyć wytrenowanych modeli do zdekodowania sygnału mowy,
- i wiele innych.

Na dzisiejszych zajęciach:
- skonfigurujesz środowisko
- skompilujesz Kaldi wraz z innymi niezbędnymi narzędziami,
- przygotujesz dane do wytrenowania bardzo prostego modelu rozpoznawania mowy
- wytrenujesz modele statystyczne - HMM-GMM oraz model neuronowy (chain)


Na początku sprawdźmy czy runtime do którego połączyliśmy się jest instancją GPU:


In [0]:
!nvidia-smi
!tar xaf drive/My\ Drive/Colab\ Notebooks/kaldi.tar.gz

Jeśli output polecenia nvidia-smi to:

```
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

```
należy zmienić instancję na instancję GPU (`Menu Runtime -> Change runtime type -> GPU`). Uruchom polecenie raz jeszcze żeby potwierdzić, że jesteś na instancji GPU.

Następnie sklonujmy repozytorium Kaldi:

In [0]:
!git clone https://github.com/kaldi-asr/kaldi.git

Aby zainstalować Kaldi, należy:
- zainstalować/skompilować zewnętrzne narzędzia (`kaldi/tools/INSTALL`)
- skompilować kod Kaldi (`kaldi/src/INSTALL`)

Zacznijmy od narzędzi:

In [0]:
!cat kaldi/tools/INSTALL

Według instrukcji skrypt `kaldi/tools/extras/check_dependencies.sh` wylistuje pakiety które należy doinstalować, sprawdźmy więc:

In [0]:
!kaldi/tools/extras/check_dependencies.sh

Jak miło - dostaliśmy gotowe polecenia do instalacji wszystkich potrzebnych pakietów. Instalujemy:
- `automake, autoconf` - narzędzia do automatycznego tworzenia plików Makefile
- `sox` - bardzo użyteczne narzędzie do analizy i przetwarzania plików audio
- `libtool` - narzędzie do linkowania binarek i bibliotek
- `subversion` - inaczej SVN (poprzednik gita) 
- `mkl` - zestaw bibliotek do algebry numerycznej od Intela
- Od siebie dodam jeszcze pakiet `flac` który będzie nam potrzebny do rozpakowania nagrań mowy w dalszej części tutoriala oraz `tree` do wyświetlania ładnie sformatowanych drzewek katalogów.

Warning o złej wersji pythona można zignorować.

In [0]:
!sudo apt-get install automake autoconf sox libtool subversion flac tree
!kaldi/tools/extras/install_mkl.sh

Po zainstalowaniu pakietów z repozytoriów, resztę należy niestety skompilować. Trochę to potrwa. Zainstalujemy:
- OpenFST - biblioteka do obsługi automatów skończenie stanowych
- Sctk - narzędzie do obliczania wyników ewaluacji

`-j2` spowoduje uruchomienie kompilacji w dwóch procesach.

In [0]:
!cd kaldi/tools && make -j2

Narzędzia są skompilowane, zaczynamy więc właściwą część instalacji. Sprawdźmy instrukcję:

In [0]:
!cat kaldi/src/INSTALL

W pierwszej kolejności musimy wygenerować skrypty instalacyjne:

In [0]:
!cd kaldi/src && ./configure --shared

Po wygenerowaniu skryptów budujących Kaldi, uruchamiamy je. Znów - chwilę to potrwa.

In [0]:
!cd kaldi/src && make depend -j2 && make -j2

Instalacja Kaldi powinna być teraz gotowa do użycia. Sprawdźmy w takim razie co znajduje się w pakiecie:

In [0]:
!tree -L 2 -d kaldi/

- `kaldi/tools` - zewnętrzne toole niezbędne do działania Kaldi
- `kaldi/src` - “właściwy” kod Kaldi. Większość z programów ma jedną, dobrze określoną funkcję, co pozwala na łatwe składanie systemu ASR z różnych dostępnych klocków.
- `kaldi/egs` - “recepty trenujące” powiązane z konkretnymi zadaniami. Receptę uruchamia się plikiem `run.sh`.
- `kaldi/egs/$nazwa_recepty/$numer_recepty/steps` - uniwersalne skrypty wykonujące duże, konkretne zadania, np. zamianę plików wav na cechy MFCC, trening modelu monofonowego itp.
- `kaldi/egs/$nazwa_recepty/$numer_recepty/local` - skrypty i pliki przygotowane pod konkretną receptę (np. przetwarzające dane wejściowe)

Receptą którą wykorzystamy do wytrenowania prostego modelu rozpoznającego jezyk angielski jest recepta mini_librispeech. Jej zmodyfikowana pod Google Colab wersja, wraz z tymże notebookiem jest dostępna w repozytorium, które należy sklonować:

In [0]:
!git clone https://github.com/tvarog/kaldi-minilibrispeech-tutorial.git

Następnie wkopiujmy receptę bezpośrednio do katalogu `kaldi/egs`:

In [0]:
cp -rvf kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/ kaldi/egs/

Jesteśmy gotowi do uruchomienia recepty. Receptę uruchamia się skryptem `run.sh`. Dla celów pokazowych została ona zmodyfikowana w taki sposób, żeby zatrzymać się po każdym wykonanym kroku. Konkretny krok wywołuje się parametrem `--stage $numer_kroku`.

Przyjrzyjmy się jak wygląda pierwszy krok:
```
if [ $stage -eq -1 ]; then
  for part in dev-clean-2 train-clean-5; do
    local/download_and_untar.sh $data $data_url $part
  done
fi
```
W skrócie - ściągamy i rozpakowujemy korpus mowy trenujący i deweloperski. Jest to tylko niewielki wycinek korpusu librispeech, okrojony dla celów demonstracyjnych.

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage -1

Sprawdźmy co wypakowaliśmy:

In [0]:
!tree kaldi/egs/mini_librispeech_tutorial/s5/corpus | head -n40

W kolejnym kroku ściągniemy korpusy tekstowe:
```
if [ $stage -eq 0 ]; then
  local/download_lm.sh $lm_url $data data/local/lm
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 0

Podejrzyjmy ściągnięte dane, najpierw model języka:

In [0]:
!zcat kaldi/egs/mini_librispeech_tutorial/s5/corpus//3-gram.arpa.gz | head -n40

Następnie prunowany (zmniejszony) model: 

In [0]:
!zcat kaldi/egs/mini_librispeech_tutorial/s5/corpus/3-gram.pruned.3e-7.arpa.gz | head -n40

Następnie leksykon:

In [0]:
!head -n40 kaldi/egs/mini_librispeech_tutorial/s5/corpus/librispeech-lexicon.txt

W kolejnym kroku bierzemy surowe, ściągnięte dane i przygotowujemy je do wykorzystania w Kaldi:
```
if [ $stage -eq 1 ]; then
  # format the data as Kaldi data directories
  for part in dev-clean-2 train-clean-5; do
    # use underscore-separated names in data directories.
    local/data_prep.sh $data/LibriSpeech/$part data/$(echo $part | sed s/-/_/g)
  done

  # take only 100 first samples from dev_clean_2 to speed-up model evaluation
  cp -rf data/dev_clean_2 data/dev_clean_2.bak
  head -n100 data/dev_clean_2.bak/wav.scp > data/dev_clean_2/wav.scp
  ./utils/fix_data_dir.sh data/dev_clean_2

  local/prepare_dict.sh --stage 3 --nj $nj --cmd "$train_cmd" \
    data/local/lm data/local/lm data/local/dict_nosp

  utils/prepare_lang.sh data/local/dict_nosp \
    "<UNK>" data/local/lang_tmp_nosp data/lang_nosp

  local/format_lms.sh --src-dir data/lang_nosp data/local/lm
  # Create ConstArpaLm format language model for full 3-gram and 4-gram LMs
  utils/build_const_arpa_lm.sh data/local/lm/lm_tgmed.arpa.gz \
    data/lang_nosp data/lang_nosp_test_tgmed
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 1

Finalnie przygotowane dane znajdują się w katalogu `data`, sprawdźmy co leży w środku:

In [0]:
!tree kaldi/egs/mini_librispeech_tutorial/s5/data

Sprawdźmy najpierw korpus trenujący. W pliku `text` znajdują się tzw. referencje, czyli transkrypcje mowy:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/text | head -n20

W pliku `wav.scp` znajdują się ścieżki do plików audio:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/wav.scp | head -n20

W pliku `spk2utt` znajdują się mappingi speaker -> ID pliku 

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/spk2utt | head -n20

Odwrotny mapping znajduje się w pliku `utt2spk`:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/utt2spk | head -n20

Mapping speaker -> płeć znajduje się w `spk2gender`:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/spk2gender | head -n20

Dane fonetyczne znajdują się z kolei w `data/local/dict_nosp`:

In [0]:
!head -n20 kaldi/egs/mini_librispeech_tutorial/s5/data/local/dict_nosp/*

Kolejnym krokiem jest zamiana plików audio na cechy sygnału:
```
if [ $stage -eq 2 ]; then
  mfccdir=mfcc

  for part in dev_clean_2 train_clean_5; do
    steps/make_mfcc.sh --cmd "$train_cmd" --nj $nj data/$part exp/make_mfcc/$part $mfccdir
    steps/compute_cmvn_stats.sh data/$part exp/make_mfcc/$part $mfccdir
  done

  # Get the shortest 500 utterances first because those are more likely
  # to have accurate alignments.
  utils/subset_data_dir.sh --shortest data/train_clean_5 500 data/train_500short
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 2

Wygenerowane cechy MFCC znajdują się w katalogu `mfcc`, podejrzyjmy też logi w `exp/make_mfcc/` 

In [0]:
!ls -l kaldi/egs/mini_librispeech_tutorial/s5/mfcc
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/make_mfcc/dev_clean_2/make_mfcc_dev_clean_2.1.log

Mamy gotowe już wszystkie niezbędne do wytrenowania systemu dane:
- cechy sygnału (czyli wejście do modelu akustycznego)
- słownik fonetyczny (czyli połączenie wejścia do modelu językowego i wyjscia z modelu akustycznego)
- model języka

Zacznijmy od wytrenowania modelu monofonowego:
```
if [ $stage -eq 3 ]; then
  steps/train_mono.sh --boost-silence 1.25 --nj $nj --cmd "$train_cmd" \
    data/train_500short data/lang_nosp exp/mono
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/mono exp/mono/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode.sh --nj $nj --cmd "$decode_cmd" exp/mono/graph_nosp_tgsmall \
        data/$test exp/mono/decode_nosp_tgsmall_$test
    done
  )

  steps/align_si.sh --boost-silence 1.25 --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/mono exp/mono_ali_train_clean_5
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 3

Sprawdźmy output z treningu:

In [0]:
!tree kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/

Interesuje nas to jak radzi sobie wytrenowany model w praktyce. Zerknijmy na logi z dekodera i porównajmy wynik z referencjami z `data/dev_clean_2/text`


In [0]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log

Sprawdźmy też ile wynosi Word Error Rate:

In [0]:
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0

Cóż, rozpoznajemy dobrze co drugie słowo. Nie najlepiej, ale pamiętajmy, że wytrenowaliśmy bardzo prosty model korzystając z bardzo małej ilości danych.

Korzystając z modelu monofonowego, wytrenujmy lepszy model - model trifonowy:
```
if [ $stage -eq 4 ]; then
  steps/train_deltas.sh --boost-silence 1.25 --cmd "$train_cmd" \
    2000 10000 data/train_clean_5 data/lang_nosp exp/mono_ali_train_clean_5 exp/tri1

  # decode using the tri1 model
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/tri1 exp/tri1/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode.sh --nj $nj --cmd "$decode_cmd" exp/tri1/graph_nosp_tgsmall \
      data/$test exp/tri1/decode_nosp_tgsmall_$test
      steps/lmrescore.sh --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri1/decode_nosp_{tgsmall,tgmed}_$test
      steps/lmrescore_const_arpa.sh \
        --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri1/decode_nosp_{tgsmall,tgmed}_$test
    done
  )

  steps/align_si.sh --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/tri1 exp/tri1_ali_train_clean_5
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 4

Tym razem sprawdzamy wytrenowany model akustyczny korzystając z małego i średniego modelu języka (tgsmall, tgmed):

In [0]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri1/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri1/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri1/decode_nosp_tgmed_dev_clean_2/wer_10_0.0

Jest dużo lepiej, ale wciąż kiepsko.

Wytrenujmy kolejny model, model trifonowy z cechami zdekorelowanymi przy użyciu LDA:
```
if [ $stage -eq 5 ]; then
  steps/train_lda_mllt.sh --cmd "$train_cmd" \
    --splice-opts "--left-context=3 --right-context=3" 2500 15000 \
    data/train_clean_5 data/lang_nosp exp/tri1_ali_train_clean_5 exp/tri2b

  # decode using the LDA+MLLT model
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/tri2b exp/tri2b/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode.sh --nj $nj --cmd "$decode_cmd" exp/tri2b/graph_nosp_tgsmall \
        data/$test exp/tri2b/decode_nosp_tgsmall_$test
      steps/lmrescore.sh --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri2b/decode_nosp_{tgsmall,tgmed}_$test
      steps/lmrescore_const_arpa.sh \
        --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri2b/decode_nosp_{tgsmall,tgmed}_$test
    done
  )

  # Align utts using the tri2b model
  steps/align_si.sh  --nj $nj --cmd "$train_cmd" --use-graphs true \
    data/train_clean_5 data/lang_nosp exp/tri2b exp/tri2b_ali_train_clean_5
fi
``` 

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 5

Sprawdźmy wyniki:

In [0]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri2b/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri2b/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri2b/decode_nosp_tgmed_dev_clean_2/wer_10_0.0

Znów poprawa.

Dodajmy dodatkową adaptację modelu per speaker (SAT):
```
if [ $stage -eq 6 ]; then
  steps/train_sat.sh --cmd "$train_cmd" 2500 15000 \
    data/train_clean_5 data/lang_nosp exp/tri2b_ali_train_clean_5 exp/tri3b

  # decode using the tri3b model
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/tri3b exp/tri3b/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode_fmllr.sh --nj $nj --cmd "$decode_cmd" \
        exp/tri3b/graph_nosp_tgsmall data/$test \
        exp/tri3b/decode_nosp_tgsmall_$test
      steps/lmrescore.sh --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri3b/decode_nosp_{tgsmall,tgmed}_$test
      steps/lmrescore_const_arpa.sh \
        --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri3b/decode_nosp_{tgsmall,tgmed}_$test
    done
  )
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 6

Sprawdźmy wyniki:

In [0]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_nosp_tgmed_dev_clean_2/wer_10_0.0

Wyniki zaczynają być niezłe.

Do tej pory zakładaliśmy, że różne wymowy tych samych słów mają identyczne prawdopodobieństwo wystąpienia. Nie jest to prawda. W poniższym kroku sprawdzimy jaki jest rozkład różnych wymów w korpusie trenującym.
```
if [ $stage -eq 7 ]; then
  steps/get_prons.sh --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/tri3b
  utils/dict_dir_add_pronprobs.sh --max-normalize true \
    data/local/dict_nosp \
    exp/tri3b/pron_counts_nowb.txt exp/tri3b/sil_counts_nowb.txt \
    exp/tri3b/pron_bigram_counts_nowb.txt data/local/dict

  utils/prepare_lang.sh data/local/dict \
    "<UNK>" data/local/lang_tmp data/lang

  local/format_lms.sh --src-dir data/lang data/local/lm

  utils/build_const_arpa_lm.sh \
    data/local/lm/lm_tgmed.arpa.gz data/lang data/lang_test_tgmed

  steps/align_fmllr.sh --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang exp/tri3b exp/tri3b_ali_train_clean_5
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 7

Zerknijmy do zaktualizowanego słownika:

In [0]:
!grep "^WITH " kaldi/egs/mini_librispeech_tutorial/s5/data/local/dict/lexiconp.txt

W następnym kroku wykorzystamy nowe informacje o słowniku i ponownie wytrenujemy ostatni model (SAT):
```
if [ $stage -eq 7 ]; then
  steps/get_prons.sh --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/tri3b
  utils/dict_dir_add_pronprobs.sh --max-normalize true \
    data/local/dict_nosp \
    exp/tri3b/pron_counts_nowb.txt exp/tri3b/sil_counts_nowb.txt \
    exp/tri3b/pron_bigram_counts_nowb.txt data/local/dict

  utils/prepare_lang.sh data/local/dict \
    "<UNK>" data/local/lang_tmp data/lang

  local/format_lms.sh --src-dir data/lang data/local/lm

  utils/build_const_arpa_lm.sh \
    data/local/lm/lm_tgmed.arpa.gz data/lang data/lang_test_tgmed

  steps/align_fmllr.sh --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang exp/tri3b exp/tri3b_ali_train_clean_5
fi
```

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 8

Sprawdźmy wyniki:

In [0]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_tgmed_dev_clean_2/wer_10_0.0

Jest już naprawdę dobrze (biorąc pod uwagę mikroskopijny zbiór uczący). Jeśli chcemy wytrenować lepszy model pozostają nam sieci neuronowe:
```
# Train a chain model
if [ $stage -eq 9 ]; then
  local/chain/run_tdnn.sh --stage 0
fi
```
Dla zainteresowanych, opis sieci TDNN wykorzystanej w Kaldi: https://en.wikipedia.org/wiki/Time_delay_neural_network

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 9

Nie pozostaje nam nic innego niż obejrzeć wyniki:

In [0]:
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp/decode_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp/decode_tgmed_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp_online/decode_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp_online/decode_tgmed_dev_clean_2/wer_10_0.0

Spadek liczby błędów jest znaczący, daleko nam jednak do jakości oferowanej przez komercyjne systemy ASR.

---

###Zadanie 1
1. Znajdź nagranie z angielską mową.
1. Skonwertuj nagranie do pliku wav, 16kHz, mono (np. w Audacity).
1. Zuploaduj nagranie na instancję.
1. Podmień `flac.*$` w ostatniej linijce `kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/wav.scp` na ścieżkę do zuploadowanego pliku. Tip: żeby otrzymać pełną ścieżkę do pliku użyj polecenia `readlink -f PLIK`
1. Podmień tekst w ostatniej linijce `kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/wav.scp` na faktyczną zawartość nowego pliku
1. Przegeneruj cechy sygnału - MFCC (stage 2)
1. Uruchom dowolny trening (np. SAT) i sprawdź logach z dekodera co zostało rozpoznane. Wynik będzie prawdopodobnie na końcu logu `decode.2.log`
1. Jako zaliczenie, wyślij plik `decode.2.log`

###Zadanie 2
1. Przygotuj maszynę wirtualną z Ubuntu 18.04.
1. Przygotuj środowisko i skompiluj Kaldi, tak jak w omawianym dziś przykładzie.
1. Wytrenuj model Kaldi na dużym zbiorze trenującym.
1. Skorzystaj z recepty kaldi/egs/librispeech/s5.
1. Jako zaliczenie, wyślij pliki raport porównujący wyniki na minilibrispeech i librispeech.

Kilka ważnych uwag:
- w pliku `cmd.sh` należy zmienić zawartość wszystkich zmiennych na `run.pl`
- w pliku run.sh należy zmienić liczbę jobów przy każdym parametrze `--nj LICZBA` na liczbę rdzeni dostępnych na maszynie wirtualnej. Uwaga na pamięć!
- w pliku run.sh należy zmienić wszystkie wystąpienia `tglarge` na `tgmed`. Na zwykłych PC model językowy `tglarge` może być zbyt duży do skompilowania.
- tym razem skrypt run.sh nie zatrzymuje się po każdym kroku. Parametr `--stage LICZBA` kontroluje teraz tylko startowy krok.
- skończ na stage 13. 